# Тема: Профилирование пользователей. Сегментация: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import re

from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize
import pymorphy2

from gensim.models import LdaModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
                                precision_recall_curve, confusion_matrix

import warnings
warnings.simplefilter('ignore')

In [2]:
news = pd.read_csv("data/articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
users = pd.read_csv("data/users_articles.csv")
print(users.shape)
users.head(3)

(8000, 2)


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [46]:
target = pd.read_csv("data/users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [201]:
morph = pymorphy2.MorphAnalyzer()

stopword_ru = stopwords.words('russian')

with open('data/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [6]:
def clean_text(text):
    '''
    очистка текста
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [35]:
%%time

news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 17.7 s, sys: 152 ms, total: 17.9 s
Wall time: 17.9 s


In [37]:
%%time

news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 3min 16s, sys: 23.9 ms, total: 3min 16s
Wall time: 3min 16s


In [42]:
texts = [t for t in news['title'].values]

common_dictionary = Dictionary(texts)

common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [43]:
%%time

lda = LdaModel(common_corpus,
               num_topics=25,
               id2word=common_dictionary)

CPU times: user 19.5 s, sys: 8 ms, total: 19.5 s
Wall time: 19.5 s


In [204]:
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.17510127),
 (6, 0.081591085),
 (8, 0.092131495),
 (18, 0.16132079),
 (21, 0.46892956)]

In [208]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)

topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: правительство регион область товар федеральный рф совет
topic_1: выяснить авария школа университет пострадать высота источник
topic_2: журнал операция риск армия всё удаться обнаружить
topic_3: космос планета земля сенатор сигнал атмосферный отель
topic_4: взрыв агентство экономика помощь освобождение бизнесмен медицинский
topic_5: газ день турция фестиваль власть турецкий россия
topic_6: украина ракета россия решение власть вопрос доход
topic_7: планета остров производитель форум nn израиль статья
topic_8: китай китайский ресурс кровь иран чёрный японский
topic_9: млрд рубль млн рост цена составить уровень
topic_10: рак берег доход мышь тыс собственность завод
topic_11: следствие университет аэропорт квартира виза студент сотрудник
topic_12: британский писать лондон грунт выдать дождь евро
topic_13: россия дать nn около тыс управление погибнуть
topic_14: российский научный военный россия украина наука украинский
topic_15: путин всё nn фонд врач болезнь очень
topic_16: nn женщ

In [209]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [210]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.782402,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.016623,0.0,0.000000,0.116377
1,4896,0.000000,0.0,0.179920,0.0,0.197066,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.061065,0.000000,0.0,0.0,0.085611,0.0,0.000000,0.000000
2,4897,0.000000,0.0,0.175017,0.0,0.000000,0.0,0.081708,0.0,0.092107,...,0.000000,0.000000,0.000000,0.161294,0.0,0.0,0.468948,0.0,0.000000,0.000000
3,4898,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.252585,0.110914,0.000000,0.000000,0.0,0.0,0.027484,0.0,0.000000,0.000000
4,4899,0.450216,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.207537,0.0,0.043816,0.000000


In [211]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

### 2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [212]:
def get_user_embedding(user_articles_list, metric=np.mean):
    
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = metric(user_vector, 0)
    
    return user_vector

In [213]:
def get_metrics(metric): 

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metric), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(25)]], 
                                                        X['churn'], random_state=0)

    logreg = LogisticRegression()

    logreg.fit(X_train, y_train)

    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_test, preds)

    ix = np.argmax(fscore)

    return roc_auc, precision[ix], recall[ix], fscore[ix]


### 3. Повторить п.2, но используя уже не медиану, а max

### 5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [214]:
metrics = {'mean': np.mean, 'median': np.median, 'max': np.max}
stata = pd.DataFrame(columns=metrics.keys())

for k, v in metrics.items():
    roc_auc, precision, recall, fscore = get_metrics(v)
    stata.loc[f'roc_auc', k] = roc_auc
    stata.loc[f'precision', k] = precision
    stata.loc[f'recall', k] = recall
    stata.loc[f'fscore', k] = fscore
    
stata

,mean,median,max
roc_auc,0.923133,0.967793,0.981259
precision,0.507082,0.704225,0.790076
recall,0.730612,0.816327,0.844898
fscore,0.598662,0.756144,0.816568


### 6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

С увеличием порога вхождения все метрики качества стремительно растут.